# Collecting Combine Data, Merging and Cleaning Notebook:

This notebook is principaly using the combine_data csv file and merges the entries with draft table from 'Collecting_Data_TableDraft.ipynb'. Additionally, since the csv document has a lot of incomplete data, we use **Pro-Football-Reference** in order to generate additional combine stats. Since the combine data for each player is hidden from the *soup document*, we will get to use **Chrome's webdriver** to scrap the combine table and add it to the draft table.

Finally, we clean and re-arrange the dataframe so that it is readable and contains only the data we care for.

In [1]:
import Collecting_Data_TableCombine_and_Cleaning
import importlib
importlib.reload(Collecting_Data_TableCombine_and_Cleaning)
from Collecting_Data_TableCombine_and_Cleaning import *

In [2]:
# import tables
combine_table = pd.read_csv('data_sets/combine_data_since_2000.csv')
td = pd.read_csv('data_sets/td.csv')

## 1) Collecting Combine Data

We collect players that were drafted between the years 2000 and 2014. Since a lot of WR draftees have various positions coming from college (not just college WRs), we make sure to include all players.

In [3]:
print(combine_table['Year'].unique())
print(combine_table['Pos'].unique())

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018]
['OLB' 'RB' 'OT' 'TE' 'CB' 'K' 'P' 'FS' 'OG' 'ILB' 'DE' 'SS' 'DT' 'QB'
 'WR' 'C' 'FB' 'LS' 'NT' 'EDGE' 'S' 'G' 'LB' 'DB' 'OL']


In [4]:
# capture players between 2000 and 2014
combine_table = combine_table[(combine_table['Year'] <= 2014) & (combine_table['Year']>=2000)].reset_index(drop=True)

## 2) Merging Combine with Draft tables

We merge the combine table and draft table by rows if (year, round, pick) match or if (year, name) match

In [5]:
tdc = merge_combine(td, combine_table)

In [6]:
tdc.combine_method.value_counts(dropna = False)

success from merge    357
fail from merge       131
Name: combine_method, dtype: int64

This many tables couldn't be merged. Let's go back to our scrapping method to search for additional combine tables from the nfl page

In [7]:
tdc.to_csv(r'data_sets/tdc_prior_to_merge.csv', index = False)

In [8]:
tdc = pd.read_csv('data_sets/tdc_prior_to_merge.csv')

## 3) Searching for more Combine Data

For those rows that failed to merge combine data, we go back to the nfl page and scrape the tables with **webdriver**. We hope to find the appropriate table 'Combine Measurements Table' 

In [10]:
tdc = search_combine(tdc)

In [11]:
tdc.combine_method.value_counts(dropna = False)

success from merge                 357
success from search                 61
fail from merge and from search     52
fail from merge and no nfl link     18
Name: combine_method, dtype: int64

We were able to salvage a few entries and generate more combine data. Let's now verify we have no mismatches w.r.t the draft year between the entry from the draft table and the entry from the combine data 

In [12]:
mismatch_combine(tdc)

[]

In [13]:
tdc.to_csv(r'data_sets/tdc_after_merge.csv', index = False)

## 4) Cleaning Data

Now, we proceed to clean and re-organize our dataframe to be more readable. This means:
- re-organizing data in a more natural and explicit way and deleting useless columns (like draft year or some useless combine stats)
- print out the type of each column
- count how we got our data and how complete is it using nfl_table_type, cfb_table_type and combine_method


In [14]:
tdc = tdc[list_of_columns_selected]
tdc = tdc.rename(columns=rename_column)

In [15]:
tdc.dtypes

year                             int64
round                            int64
pick                             int64
player                          object
cfb_url                         object
cfb_table_name                  object
cfb_school                      object
cfb_conference                  object
cfb_class                       object
cfb_pos                         object
cfb_games                      float64
cfb_receptions                 float64
cfb_rec_yards                  float64
cfb_rec_yards_per_reception    float64
cfb_rec_td                     float64
cfb_rushes                     float64
cfb_rushing_yards              float64
cfb_rushing_yards_per_rush     float64
cfb_rushing_td                 float64
cfb_plays                      float64
cfb_yards                      float64
cfb_yards_per_play             float64
cfb_td                         float64
combine_method                  object
combine_draft_age                int64
combine_ht               

In [21]:
pd.crosstab(tdc.nfl_table_name, tdc.cfb_table_name)

cfb_table_name,cfb link is missing,receiving & rushing table,receiving & rushing table inverted,receiving & rushing table webdriver,receiving & rushing table webdriver inverted
nfl_table_name,,,,,
receiving & rushing table,52,358,6,2,7
receiving & rushing table inverted,0,0,1,0,2
receiving & rushing table manually,0,1,0,0,2
receiving & rushing table webdriver,14,32,4,3,1
receiving & rushing table webdriver inverted,1,1,1,0,0


In [22]:
pd.crosstab(tdc.nfl_table_name, tdc.combine_method)

combine_method,fail from merge and from search,fail from merge and no nfl link,success from merge,success from search
nfl_table_name,,,,
receiving & rushing table,38,18,313,56
receiving & rushing table inverted,0,0,3,0
receiving & rushing table manually,2,0,1,0
receiving & rushing table webdriver,12,0,37,5
receiving & rushing table webdriver inverted,0,0,3,0


In [25]:
no_cfb_combine = tdc.loc[(tdc.cfb_table_name == 'cfb link is missing') & ( (tdc.combine_method == 'fail from merge and from search') | (tdc.combine_method == 'fail from merge and no nfl link') )]
no_combine = tdc.loc[( (tdc.combine_method == 'fail from merge and from search') | (tdc.combine_method == 'fail from merge and no nfl link') )]
no_cfb = tdc.loc[tdc.cfb_table_name == 'cfb link is missing']

In [27]:
print('Total number of rows', len(tdc))
print('Data with neither cfb nor combine', len(no_cfb_combine) )
print('Data with no cfb ', len(no_cfb))
print('Data with no combine ', len(no_combine))

Total number of rows 488
Data with neither cfb nor combine 25
Data with no cfb  67
Data with no combine  70


In [28]:
tdc.to_csv(r'data_sets/tdc_ready_for_export.csv', index = False)

## Others - Testing for combine data from search

Testing with a particular player what is the combine data from the search function

In [263]:
url_nfl = 'https://www.pro-football-reference.com/players/M/MartKe01.htm'
all_tables = generate_all_tables(url_nfl)
captions_nfl = find_captions_from_list_of_tables(all_tables)

In [258]:
table_combine = combinestats(all_tables[-1])
display(table_combine)
print('And the combine row exported:')
x, y = row_combine(table_combine)
print(x, y)
[x[0]] + list(map( lambda i: 0 if i == '' else float(i) , x[1:] ))

,year,pos,height,weight,40yd,bench,broad jump,shuttle,3 cone,vertical
0,Year,Pos,Ht,Wt,40yd,Bench,Broad Jump,Shuttle,3Cone,Vertical
1,2012,WR,71,188,4.42,13,122,4.13,6.85,39.5


And the combine row exported:
['WR', 71.0, 188.0, 4.42, 39.5, 13.0, 122.0, 6.85, 4.13, 2012.0] success from search


['WR', 71.0, 188.0, 4.42, 39.5, 13.0, 122.0, 6.85, 4.13, 2012.0]